# ДЗ №2

Были взяты данные из амазона, категория -- музыкальные инструменты.

In [1]:
import json
from tqdm.notebook import tqdm

In [2]:
reviewdata_json = []
with open("Musical_Instruments.json", "r", encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        reviewdata_json.append(json.loads(line))

  0%|          | 0/1512530 [00:00<?, ?it/s]

In [3]:
reviewdata_json[8888]

{'overall': 5.0,
 'verified': True,
 'reviewTime': '08 25, 2015',
 'reviewerID': 'AYLKVGDND9FJJ',
 'asin': 'B000068OHN',
 'reviewerName': 'scott davis',
 'reviewText': 'Just started using the mics. So far so good. They arrived on time and are easy to set up. I will provide additional feed back as they are used more.',
 'summary': 'So far so good. They arrived on time and are easy to ...',
 'unixReviewTime': 1440460800}

In [3]:
metadata_json = []
with open("meta_Musical_Instruments.json", "r", encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        metadata_json.append(json.loads(line))

  0%|          | 0/120310 [00:00<?, ?it/s]

In [4]:
reviewdata_asins = [review["asin"] for review in tqdm(reviewdata_json)]

  0%|          | 0/1512530 [00:00<?, ?it/s]

In [5]:
reviewdata_asins = set(reviewdata_asins)

In [6]:
len(reviewdata_asins)

112222

## 1. Способы

- Посмотреть на названия категорий, перечисленные в метаданных, отфильтровать их и искать их упоминания в отзывах.
- Сделать то же самое, но с названиями продуктов (тоже перечисленных в метаданных).
- Составить список базовых инструментов, расширить их при помощи эмбеддингов, обученных на тех же отзывах. Искать их упоминание в отзывах.

Здесь будет использован гибрид из первого и второго пунктов.

## 2. Реализация

In [7]:
asin2product = {}
for product in tqdm(metadata_json):
    if product["asin"] in reviewdata_asins:
        asin2product[product["asin"]] = {k:v for k,v in product.items() if k!="asin"}

  0%|          | 0/120310 [00:00<?, ?it/s]

In [8]:
product_names = []
for product in asin2product:
    product_names.append(asin2product[product]["title"])

In [9]:
product_names

['Wooden Percussion 2 Piece Set of 3 Inch Cricket and Inch Owl',
 'Wooden Percussion 3 Piece Set Frog, Cricket and Owl, 3 Inches',
 'Hal Leonard Vivaldi Four Seasons for Piano (Original Italian Text)',
 'The Turn of the Screw (vocal score)',
 'Suite for Organ (including the Trumpet Voluntary) [Sheet Music Book]',
 'Wiley Publishers Crocheting For Dummies Revised',
 'Nook Audio OE250 Headphones',
 'Being Here',
 'Hal Leonard The Big Klezmer Fake Book',
 'Paul Winter: Missa Gaia (Earth Mass) - SATB',
 'Music Theory Staff Paper Notebook',
 'Ukulele Chord Changer and Songbook Set',
 'Yamaha Dx7 Digital Synthesizer',
 "Jazz Pieces for the Classical Guitar: Seventeen Famous Jazz 'standards' Arranged for the Classical Guitar",
 "Alfred Alfred's Pro Audio Series: Beginning Pro Tools LE 8 (DVD)",
 "Alfred's Kid's Guitar Course, Complete Starter Pack: Everything You Need to Play Today!",
 'Alfreds Teach Yourself to Play Ukulele, Complete Starter Pack',
 'Journey - Greatest Hits: Easy Piano',
 'B

In [10]:
product_categories = []
for product in asin2product:
    product_categories.extend(asin2product[product]["category"])

In [11]:
from pattern3.text.en import singularize

In [12]:
set(product_categories)

{'Lightweight rigid EPS foam frame, high quality plastic hardware, rugged 600D nylon exterior, and durable molded rubber feet',
 'DJ Controllers',
 'Pick Holders',
 'Drum Machines',
 'Powered Cabinets',
 'China &amp; Effect',
 'Large exterior front pocket with built-in organizer for small accessories',
 'Marching Tom-Toms',
 'MIDI Controllers',
 'Soundtrack Percussion: Create a blockbuster effect with the rhythm section',
 'Saxophone',
 'Boom',
 'Drum Accessories',
 'Water and abrasion resistant padded ballistic nylon exterior',
 'Acoustic & Classical Guitar Bags & Cases',
 'Doumbeks',
 'Pan Pipes',
 'Cornets',
 'Concert Cymbals',
 'Flutes &amp; Piccolos',
 'Ultra-Quiet Velocity Sensitive Pads',
 'Band & Orchestra',
 'Requires a working Superior Drummer 2.0 installation.',
 'Studio Subwoofers',
 'Musical Instruments',
 'Sound Editing',
 'Acoustic Guitar Effects',
 'Banjo Accessories',
 'Stage Speaker Stands',
 'Made in Nepal.',
 'Full accessibility to all the buttons',
 'Every zebra ha

In [13]:
singularize("CD Players")

'CD Player'

In [14]:
product_categories_split = []
for cat in set(product_categories):
    if "&amp;" in cat:
        product_categories_split.extend([elem.strip() for elem in cat.split("&amp;")])
    elif "&" in cat:
        product_categories_split.extend([elem.strip() for elem in cat.split("&")])
    else:
        product_categories_split.append(cat.strip())

In [15]:
set(product_categories_split)

{'Lightweight rigid EPS foam frame, high quality plastic hardware, rugged 600D nylon exterior, and durable molded rubber feet',
 '',
 'DJ Controllers',
 'Pick Holders',
 'Drum Machines',
 'Powered Cabinets',
 'Large exterior front pocket with built-in organizer for small accessories',
 'Marching Tom-Toms',
 'Soundtrack Percussion: Create a blockbuster effect with the rhythm section',
 'MIDI Controllers',
 'Saxophone',
 'Boom',
 'Drum Accessories',
 'Water and abrasion resistant padded ballistic nylon exterior',
 'Cornets',
 'Doumbeks',
 'Pan Pipes',
 'Concert Cymbals',
 'Subwoofer Parts',
 'Ultra-Quiet Velocity Sensitive Pads',
 'Requires a working Superior Drummer 2.0 installation.',
 'Studio Subwoofers',
 'Musical Instruments',
 'Sound Editing',
 'Acoustic Guitar Effects',
 'Lighting Equipment',
 'Banjo Accessories',
 'Stage Speaker Stands',
 'Splitters',
 'Made in Nepal.',
 'Full accessibility to all the buttons',
 'Every zebra has a unique pattern of black and white stripes.',
 'Sl

In [16]:
product_categories_sing = [singularize(cat) for cat in set(product_categories_split)]

In [18]:
for name in product_names:
    if len(name.split()) <= 3:
        product_categories_sing.append(name)

In [19]:
product_categories_sing

['Lightweight rigid EPS foam frame, high quality plastic hardware, rugged 600D nylon exterior, and durable molded rubber foot',
 '',
 'DJ Controller',
 'Pick Holder',
 'Drum Machine',
 'Powered Cabinet',
 'Large exterior front pocket with built-in organizer for small accessory',
 'Marching Tom-Tom',
 'Soundtrack Percussion: Create a blockbuster effect with the rhythm section',
 'MIDI Controller',
 'Saxophone',
 'Boom',
 'Drum Accessory',
 'Water and abrasion resistant padded ballistic nylon exterior',
 'Cornet',
 'Doumbek',
 'Pan Pipe',
 'Concert Cymbal',
 'Subwoofer Part',
 'Ultra-Quiet Velocity Sensitive Pad',
 'Requires a working Superior Drummer 2.0 installation.',
 'Studio Subwoofer',
 'Musical Instrument',
 'Sound Editing',
 'Acoustic Guitar Effect',
 'Lighting Equipment',
 'Banjo Accessory',
 'Stage Speaker Stand',
 'Splitter',
 'Made in Nepal.',
 'Full accessibility to all the button',
 'Every zebra has a unique pattern of black and white stripes.',
 'Slide',
 'Pitch',
 'Nickel

In [20]:
import re

In [21]:
product_types = list(filter(lambda x: len(x.split()) <= 5, product_categories_sing))

In [22]:
product_types = list(map(lambda x: re.sub("[^a-zA-Z\- ]", "", x).strip(), product_types))

In [23]:
product_types

['',
 'DJ Controller',
 'Pick Holder',
 'Drum Machine',
 'Powered Cabinet',
 'Marching Tom-Tom',
 'MIDI Controller',
 'Saxophone',
 'Boom',
 'Drum Accessory',
 'Cornet',
 'Doumbek',
 'Pan Pipe',
 'Concert Cymbal',
 'Subwoofer Part',
 'Ultra-Quiet Velocity Sensitive Pad',
 'Studio Subwoofer',
 'Musical Instrument',
 'Sound Editing',
 'Acoustic Guitar Effect',
 'Lighting Equipment',
 'Banjo Accessory',
 'Stage Speaker Stand',
 'Splitter',
 'Made in Nepal',
 'Slide',
 'Pitch',
 'Nickel plated hardware',
 'In-Ear Audio Monitor',
 'Envelope follower',
 'Variety Pack',
 'Removable adjustable shoulder strap',
 'Bagpipe',
 'Speaker',
 'no-closure closure',
 'Unpowered Cabinet',
 'Portable System',
 'Pickup Cover',
 'Replacement Cartridge Needle',
 'More than  sound',
 'Conch Shell',
 'Snow Machine',
 'Complete professional audio production suite',
 'Oud',
 'Electronic Music DJ',
 'Amplifier',
 'Record your live mixes',
 'Enhancer',
 'Overdrive',
 'Multieffect',
 'Ashiko',
 'Acoustic Guitar Str

In [24]:
import spacy

In [25]:
nlp = spacy.load("en")

In [26]:
from spacy.matcher import Matcher

In [57]:
matcher = Matcher(nlp.vocab)

In [58]:
product_types = set([product.lower() for product in set(product_types) if product != ""])

In [59]:
for cat in product_types:
    if cat == "":
        pass
    else:
        pattern_full_lemma = [{"POS": "DET", "OP": "*"}, {"POS": {"IN": ["NOUN", "X"]}, "OP": "*"}]
        pattern_full = [{"POS": "DET", "OP": "*"}, {"POS": {"IN": ["NOUN", "X"]}, "OP": "*"}]
        pattern_post = []
        cat_split = cat.split()
        pattern_full.extend([{"LOWER": tok.lower()} for tok in cat_split])
        pattern_full_lemma.extend([{"LEMMA": tok.lower()} for tok in cat_split])
        pattern_post.extend([{"LOWER": tok.lower()} for tok in cat_split])
        pattern_post.append({"POS": {"IN": ["NOUN", "X"]}, "OP": "*"})
        matcher.add(cat, [pattern_full_lemma,
                              pattern_full,
                              pattern_post])

In [65]:
product_types

{'pedal steel guitar',
 'magma',
 'drum workshop lanyard',
 'practice chanters',
 'soundtech vocal trainer',
 'digital drum sticks',
 'humitron bass humidifier',
 'high quality real guitar stand',
 'bach prelude trumpet',
 'rme adi-',
 'kyser lem-oil',
 'taylor tz case',
 'concert percussion accessory',
 'kluson plate nickelplastic',
 'rugged nylon exterior',
 'peavey feedback ferret',
 'monitors speaker',
 'shofar',
 'pick guard',
 'hartland train whistle',
 'slipmat',
 'slide',
 'high',
 'peg dope',
 'midisoft guitar tool',
 'ay groove tubes',
 'music write',
 'fender christmas stocking',
 'atlas sound ds',
 'hohner bass harmonica',
 'soulo wired microphone',
 'toca t-ss shaker',
 'mounted tambourine',
 'adler alto recorder',
 'inch mirror ball',
 'paiste cymbal cleaner',
 'imusic tools',
 'flute',
 'kubudo stand',
 'fbt maxx a',
 'sabian crisis kit',
 'avlex hs-bk-tc',
 'lovepedal kalamazoo gold',
 'saroyan nefertiti zil',
 'triangle',
 'portable electronic keyboard',
 'powered cabi

In [66]:
doc = nlp("The pedal steel guitar")

In [67]:
for match_id, start, end in matcher(doc):
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(match_id, string_id, start, end, span.text)

15348571063247275712 pedal 0 2 The pedal
15348571063247275712 pedal 1 2 pedal
5357052437301627421 pedal steel guitar 0 4 The pedal steel guitar
5357052437301627421 pedal steel guitar 1 4 pedal steel guitar
15348571063247275712 pedal 1 3 pedal steel
13533102915073649304 guitar 2 4 steel guitar
13533102915073649304 guitar 3 4 guitar
15348571063247275712 pedal 1 4 pedal steel guitar


In [139]:
doc = nlp(reviewdata_json[8890]["reviewText"])

In [62]:
print("TYPE\tSTART\tEND\tSPAN\tBIGRAM_LEFT\tBIGRAM_RIGHT")
for match_id, start, end in matcher(doc):
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    if start != 0:
        span_left = doc[start-1:end]
    else:
        span_left = span
    span_right = doc[start:end+1]
    print(string_id, start, end, span.text, span_left.text, span_right.text, sep="\t")

TYPE	START	END	SPAN	BIGRAM_LEFT	BIGRAM_RIGHT
keyboard	0	2	The Keyboard	The Keyboard	The Keyboard
keyboard	1	2	Keyboard	The Keyboard	Keyboard


In [51]:
reviewdata_json[8889]

{'overall': 5.0,
 'verified': False,
 'reviewTime': '08 21, 2015',
 'reviewerID': 'A9II06CGNNWRC',
 'asin': 'B000068OHN',
 'reviewerName': 'Alma Burgos',
 'reviewText': 'I love it...excellent for the praise team of my church...good sound!',
 'summary': 'Good for church...',
 'unixReviewTime': 1440115200}

In [63]:
import random

In [68]:
random.shuffle(reviewdata_json)

In [69]:
df_dict = {"reviewID": [],
           "ruleType": [],
           "start": [],
           "end": [],
           "span": [],
           "left": [],
           "right": []}
for review in tqdm(reviewdata_json[:10000]):
    try:
        doc = nlp(re.sub("[^a-zA-Z\- ]", "", review["reviewText"]))
        for match_id, start, end in matcher(doc):
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]
            if start == 0:
                left = doc[start:start]
            elif start == 1:
                left = doc[start-1:start]
            else:
                left = doc[start-2:start]
            right = doc[end:end+2]
            df_dict["reviewID"].append(review["reviewerID"]+review["asin"])
            df_dict["ruleType"].append(string_id)
            df_dict["start"].append(start)
            df_dict["end"].append(end)
            df_dict["span"].append(span.text)
            df_dict["left"].append(left.text)
            df_dict["right"].append(right.text)
    except:
        print("oops")
        pass

  0%|          | 0/10000 [00:00<?, ?it/s]

oops
oops
oops
oops
oops
oops
oops


In [71]:
import pandas as pd

In [72]:
df = pd.DataFrame.from_dict(df_dict)

In [110]:
df.head()

,reviewID,ruleType,start,end,span,left,right
0,A1F893Z6409CXTB000B6FADU,band,19,21,a band,son has,and his
1,A1F893Z6409CXTB000B6FADU,band,20,21,band,has a,and his
2,A3UALD7ZJ1C7K1B0002EWTYE,sound,1,3,THE SOUND,LOVE,sound is
3,A3UALD7ZJ1C7K1B0002EWTYE,sound,2,3,SOUND,LOVE THE,sound is
4,A3UALD7ZJ1C7K1B0002EWTYE,sound,3,4,sound,THE SOUND,is so


Попробуем только с продуктами, у которых название в одно слово.

In [73]:
df_oneword = []
for row in df.values:
    if len(row[1].split()) == 1:
        df_oneword.append(row)

In [74]:
df_oneword = pd.DataFrame(df_oneword)

In [75]:
df_oneword

,0,1,2,3,4,5,6
0,A1F893Z6409CXTB000B6FADU,band,19,21,a band,son has,and his
1,A1F893Z6409CXTB000B6FADU,band,20,21,band,has a,and his
2,A3UALD7ZJ1C7K1B0002EWTYE,sound,1,3,THE SOUND,LOVE,sound is
3,A3UALD7ZJ1C7K1B0002EWTYE,sound,2,3,SOUND,LOVE THE,sound is
4,A3UALD7ZJ1C7K1B0002EWTYE,sound,3,4,sound,THE SOUND,is so
...,...,...,...,...,...,...,...
52067,A1BG1F78F3HQZFB00NS4HFZI,very,6,9,singing bowls Very,my favorite,nice looking
52068,A1BG1F78F3HQZFB00NS4HFZI,very,7,9,bowls Very,favorite singing,nice looking
52069,A1BG1F78F3HQZFB00NS4HFZI,very,8,9,Very,singing bowls,nice looking
52070,A1BG1F78F3HQZFB00NS4HFZI,sound,18,19,sounds,-lasting,


In [92]:
df_oneword = df_oneword[df_oneword[1] != "very"]

In [98]:
df_oneword = df_oneword[df_oneword[1] != "at"]

In [99]:
grams = []
for row in df_oneword.values:
    grams.append(row[5].split()+[row[4]]+row[6].split())
    grams[-1] = grams[-1][1:4]

In [100]:
grams[14]

['better', 'delays', 'but']

In [101]:
import nltk
from nltk.collocations import *

In [102]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(grams)

Что у нас получается:

In [103]:
finder.score_ngrams(bigram_measures.likelihood_ratio)

[(('this', 'guitar'), 643.7489164842145),
 (('the', 'sound'), 624.5683475129292),
 (('sounds', 'great'), 555.6484223895444),
 (('acoustic', 'guitar'), 554.6958935121438),
 (('high', 'quality'), 523.0984298263558),
 (('these', 'strings'), 491.25554245104166),
 (('gig', 'bag'), 490.2562343667994),
 (('sound', 'quality'), 456.94709292567086),
 (('pop', 'filter'), 451.13638401579345),
 (('it', 'sounds'), 431.125753397282),
 (('my', 'guitar'), 430.6069466898829),
 (('pick', 'up'), 378.5175287569316),
 (('music', 'stand'), 353.81657996881205),
 (('the', 'neck'), 345.8976428118609),
 (('part', 'of'), 325.8492109503621),
 (('this', 'pedal'), 317.00743649596524),
 (('the', 'strings'), 301.6222426919056),
 (('shoulder', 'rest'), 300.15157056658495),
 (('mic', 'stand'), 282.8725451642057),
 (('an', 'acoustic'), 281.10783013188984),
 (('and', 'the'), 272.62585121109555),
 (('mini video', '-blockdivinput'), 266.34714648146144),
 (('the', 'guitar'), 256.49114288873875),
 (('acoustic', 'guitars'), 25

In [104]:
finder.score_ngrams(bigram_measures.pmi)

[(('-- bedroom', 'bathroom'), 17.036957006502096),
 (('-- crystal', '-clear'), 17.036957006502096),
 (('--Jumbo', '-SizedpBJYLHGOrefcmcrarpdrvwtxtieUTFClassic'),
  17.036957006502096),
 (('--Male', '--Female'), 17.036957006502096),
 (('--bit', '-Linear'), 17.036957006502096),
 (('--feet', '-Closed'), 17.036957006502096),
 (('--v', '--A'), 17.036957006502096),
 (('-fingered', 'rubber mutes'), 17.036957006502096),
 (('ATE', 'Congratulations'), 17.036957006502096),
 (('AVOID', 'THIS BOW'), 17.036957006502096),
 (('Aggressive', 'This thing picks'), 17.036957006502096),
 (('Artist', 'series guitars'), 17.036957006502096),
 (('BAG', 'REALLY'), 17.036957006502096),
 (('BANJO', 'ARE'), 17.036957006502096),
 (('BLURB', 'The multitrack recording'), 17.036957006502096),
 (('BREW', 'PASSIVE ANTENNA TUNER'), 17.036957006502096),
 (('Besides', 'that drum'), 17.036957006502096),
 (('Black K-', 'ItemsOrderaTRANSPORE'), 17.036957006502096),
 (('Boost', 'switchesThis pedal'), 17.036957006502096),
 (('Bo

In [105]:
finder.score_ngrams(bigram_measures.student_t)

[(('the', 'sound'), 15.302103297362148),
 (('this', 'guitar'), 13.376874924373542),
 (('my', 'guitar'), 11.464539716759687),
 (('the', 'guitar'), 11.076957895099358),
 (('the', 'strings'), 10.882987684336562),
 (('acoustic', 'guitar'), 10.862671088465671),
 (('sound', 'quality'), 10.310335094112407),
 (('sounds', 'great'), 10.190928201298444),
 (('the', 'neck'), 9.736423904815325),
 (('the', 'case'), 9.546630383564418),
 (('these', 'strings'), 9.245569326593982),
 (('it', 'sounds'), 9.191476882581311),
 (('high', 'quality'), 9.074769418468176),
 (('sound', 'is'), 8.849356535702526),
 (('this', 'pedal'), 8.708415235204063),
 (('strings', 'are'), 8.08829942731172),
 (('case', 'is'), 7.903342542544148),
 (('part', 'of'), 7.752117875176991),
 (('the', 'stand'), 7.633501975160033),
 (('the', 'bridge'), 7.581578448901239),
 (('the', 'instrument'), 7.269308417986559),
 (('my', 'guitars'), 7.240072189406499),
 (('pick', 'up'), 7.188232461953629),
 (('the', 'keyboard'), 7.150629773121601),
 (('

Кажется, нам нужно больше данных. Берем полный датафрейм.

In [115]:
df = df[df["ruleType"] != "very"]
df = df[df["ruleType"] != "at"]
df = df[df["ruleType"] != "all"]

In [140]:
df.head()

,reviewID,ruleType,start,end,span,left,right
0,A1F893Z6409CXTB000B6FADU,band,19,21,a band,son has,and his
1,A1F893Z6409CXTB000B6FADU,band,20,21,band,has a,and his
2,A3UALD7ZJ1C7K1B0002EWTYE,sound,1,3,THE SOUND,LOVE,sound is
3,A3UALD7ZJ1C7K1B0002EWTYE,sound,2,3,SOUND,LOVE THE,sound is
4,A3UALD7ZJ1C7K1B0002EWTYE,sound,3,4,sound,THE SOUND,is so


In [120]:
grams = []
for row in tqdm(df.values):
    grams.append(row[5].split()+[row[4]]+row[6].split())
    grams[-1] = grams[-1][1:4]

  0%|          | 0/47823 [00:00<?, ?it/s]

## 3. Подсчитать при помощи 3-х метрик

In [121]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(grams)

In [124]:
finder.score_ngrams(bigram_measures.likelihood_ratio)

[(('this', 'guitar'), 644.7497678739871),
 (('the', 'sound'), 630.590628785704),
 (('acoustic', 'guitar'), 561.6809845805928),
 (('sounds', 'great'), 556.5476842019748),
 (('high', 'quality'), 527.629398574768),
 (('gig', 'bag'), 493.56887620711564),
 (('these', 'strings'), 493.5451804588612),
 (('sound', 'quality'), 462.8684621142481),
 (('pop', 'filter'), 449.5321757341775),
 (('it', 'sounds'), 430.69485465277256),
 (('my', 'guitar'), 418.5786671605143),
 (('pick', 'up'), 381.8359806833614),
 (('music', 'stand'), 355.8178897294675),
 (('the', 'neck'), 348.1646478782603),
 (('part', 'of'), 325.9228351825509),
 (('this', 'pedal'), 318.01202134975927),
 (('the', 'strings'), 305.10431062636525),
 (('shoulder', 'rest'), 302.1210486657651),
 (('and', 'the'), 287.27920465582184),
 (('mic', 'stand'), 285.6430522753584),
 (('an', 'acoustic guitar'), 272.7266162621787),
 (('mini video', '-blockdivinput'), 267.4143145278299),
 (('an', 'acoustic'), 267.3537714197924),
 (('the', 'guitar'), 259.54

In [125]:
finder.score_ngrams(bigram_measures.pmi)

[(('-- bedroom', 'bathroom'), 17.096169511668805),
 (('-- crystal', '-clear'), 17.096169511668805),
 (('--Jumbo', '-SizedpBJYLHGOrefcmcrarpdrvwtxtieUTFClassic'),
  17.096169511668805),
 (('--Male', '--Female'), 17.096169511668805),
 (('--bit', '-Linear'), 17.096169511668805),
 (('--feet', '-Closed'), 17.096169511668805),
 (('--v', '--A'), 17.096169511668805),
 (('-X', 'series excellent quality'), 17.096169511668805),
 (('-fingered', 'rubber mutes'), 17.096169511668805),
 (('ATE', 'Congratulations'), 17.096169511668805),
 (('AVOID', 'THIS BOW'), 17.096169511668805),
 (('Aggressive', 'This thing picks'), 17.096169511668805),
 (('Artist', 'series guitars'), 17.096169511668805),
 (('BAG', 'REALLY'), 17.096169511668805),
 (('BANJO', 'ARE'), 17.096169511668805),
 (('BEST', 'STUDIO MONITORS'), 17.096169511668805),
 (('BLURB', 'The multitrack recording'), 17.096169511668805),
 (('BREW', 'PASSIVE ANTENNA TUNER'), 17.096169511668805),
 (('Besides', 'that drum'), 17.096169511668805),
 (('Black K-

In [126]:
finder.score_ngrams(bigram_measures.student_t)

[(('the', 'sound'), 15.348145586108336),
 (('this', 'guitar'), 13.385340082060441),
 (('my', 'guitar'), 11.403181163685701),
 (('the', 'guitar'), 11.12562458585888),
 (('the', 'strings'), 10.920929363544737),
 (('acoustic', 'guitar'), 10.881639385825569),
 (('sound', 'quality'), 10.330982470594533),
 (('sounds', 'great'), 10.193640356830604),
 (('the', 'neck'), 9.750641756295863),
 (('the', 'case'), 9.573033752009012),
 (('these', 'strings'), 9.250430013540866),
 (('it', 'sounds'), 9.19166278900128),
 (('high', 'quality'), 9.080240354046646),
 (('sound', 'is'), 8.8805327442514),
 (('this', 'pedal'), 8.713257495027422),
 (('strings', 'are'), 8.086831255159746),
 (('case', 'is'), 7.91745561051012),
 (('part', 'of'), 7.753225729830789),
 (('the', 'stand'), 7.671155327124876),
 (('the', 'bridge'), 7.59551004931564),
 (('the', 'instrument'), 7.2938331764092705),
 (('my', 'guitars'), 7.222126436733066),
 (('pick', 'up'), 7.191153514724755),
 (('the', 'keyboard'), 7.149845751545794),
 (('the'

Тест Стьюдента и LR очень похожи. PMI оказался наименее полезен.

## 4. Посмотрим какие получились самые высокие

In [136]:
instruments = ["guitar", "drums", "keyboard", "ukulele", "violin"]

In [139]:
collocs = finder.score_ngrams(bigram_measures.student_t)
for inst in instruments:
    print(inst)
    print("---")
    i = 0
    n = 0
    while n < 5:
        if inst in collocs[i][0]:
            print(" ".join(collocs[i][0]))
            n += 1
        i += 1
    print("---\n")

guitar
---
this guitar
my guitar
the guitar
acoustic guitar
a guitar
---

drums
---
the drums
my drums
drums and
electronic drums
these drums
---

keyboard
---
the keyboard
this keyboard
my keyboard
MIDI keyboard
keyboard I
---

ukulele
---
this ukulele
a ukulele
my ukulele
ukulele player
tenor ukulele
---

violin
---
this violin
the violin
my violin
a violin
violin is
---



Одну вещь точно можно было бы исправить: при поиске сущностей делать детерминатор обязательной частью. Иначе получается так, что n-граммы показывают то, что и так известно: чаще всего используются с этими словами детерминативы.

In [141]:
# made by nejenek